In [3]:
import os
import pandas as pd
import librosa
import soundfile as sf
import numpy as np
import math
import scipy.io.wavfile, scipy.signal
from scipy.spatial import distance
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#from sklearn.naive_bayes import GaussianNB
#from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

lut=pd.read_csv('../dataset/annotation.csv')

## Extract vocal only data using spleeter

In [5]:
import os
from spleeter.separator import Separator
import soundfile as sf
import librosa
# Using embedded configuration.
separator = Separator('spleeter:2stems')

# # Using custom configuration file.
# separator = Separator('/path/to/config.json')

# Use audio loader explicitly for loading audio waveform :
from spleeter.audio.adapter import AudioAdapter

audio_loader = AudioAdapter.default()
sample_rate = 44100
i=0
loc='../dataset/audio/'
for file_name in os.listdir(loc):
    if file_name.endswith(".wav"):
        name=file_name[:-4]
        waveform, _ = audio_loader.load(loc+file_name, sample_rate=sample_rate)
        #waveform,_ = librosa.load(loc+file_name, sr=44100, mono=False)

        # Perform the separation :
        prediction = separator.separate(waveform)
        sf.write('./resources/vocal_only_audio/'+name+'_vocal.wav', prediction['vocals'], 44100)
        # if i == 0:
        #     print(prediction['vocals'])
        #     break

INFO:tensorflow:Apply unet for vocals_spectrogram
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Apply unet for accompaniment_spectrogram
INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 2stems archive
INFO:spleeter:2stems model file(s) extracted
INFO:tensorflow:Restoring parameters from pretrained_models/2stems/model


## Divide audio into 1 second chunks
Chunks are labelled based on comparing the midpoint of the block to the annotation values

In [6]:
# BLOCK AUDIO
def block_audio(x,blockSize,hopSize,fs):    
    # allocate memory    
    numBlocks = math.ceil(x.size / hopSize)    
    xb = np.zeros([numBlocks, blockSize])    
    # compute time stamps    
    t = (np.arange(0, numBlocks) * hopSize) / fs   
    t_mid = t + (0.5*blockSize/fs)
    x = np.concatenate((x, np.zeros(blockSize)),axis=0)    
    for n in range(0, numBlocks):        
        i_start = n * hopSize        
        i_stop = np.min([x.size - 1, i_start + blockSize - 1])        
        xb[n][np.arange(0,blockSize)] = x[np.arange(i_start, i_stop + 1)]    
    return (xb,t,t_mid)


In [9]:
i=0
for video_id in lut['video_id'].unique():
    try:
        fp='./resources/vocal_only_audio/'+video_id+'_vocal.wav'
        #print(fp)
        x,sr = librosa.load(fp,sr=44100,mono=True)
        blockSize = int(sr * 1) # One second blocks
        hopSize = int(sr * 0.5) # half second hops

        xb,t,t_mid = block_audio(x,blockSize,hopSize,sr)
        labels=[]
        for ts in t_mid:
            for idx,row in lut[lut['video_id'] == video_id].reset_index().iterrows():
                annotated_start = row['timestamp_start']
                annotated_end = row['timestamp_end']
                if annotated_start <= ts <= annotated_end:
                    labels.append(row['scream_type'])
                    break
            if ~(annotated_start <= ts <= annotated_end):
                labels.append('no_vocals')
        # Create new dataframs
        if i == 0:
            print("Creating dataframe")
            df=pd.DataFrame()

            df['t'] = t
            df['t_mid'] = t_mid
            blocks=[]
            for blk in xb:
                blocks.append(blk)
            df.insert(0,'video_id',video_id)
            df.insert(3,'label',labels)
            df['xb'] = blocks
            i+=1
        else:
            tmp=pd.DataFrame()
            tmp['t'] = t
            tmp['t_mid'] = t_mid
            blocks=[]
            for blk in xb:
                blocks.append(blk)
            tmp['video_id'] = video_id
            tmp['label'] = labels
            tmp['xb'] = blocks
            df=df.append(tmp)
    except Exception as e:
        print(e)
out = df.to_numpy()
np.save('./resources/working_data/vocal_only_data.npy', out)
    #df.to_csv('./resources/working_data/'+video_id+'.csv',header=True, index=False,encoding='utf-8-sig',sep='\t')


/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


[Errno 2] No such file or directory: './resources/vocal_only_audio/4600fGWcn9o_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/GLu-E42-RmA_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/ebSxxr726_8_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/iVvXB-Vwnco_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/F3A_3c882us_vocal.wav'


/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


[Errno 2] No such file or directory: './resources/vocal_only_audio/1gpfzCxiQ-A_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/B3F10hXdmQY_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/09KScSe4hIc_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/Dhfy9TPga-c_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/BR2kSva4NT8_vocal.wav'


/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


...

/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/cor

[Errno 2] No such file or directory: './resources/vocal_only_audio/HKWqzjQAv14_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/-xYZM04JxnQ_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/FNdC_3LR2AI_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/74nTzbgDGWM_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/HUUBI7RJtr8_vocal.wav'


/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


[Errno 2] No such file or directory: './resources/vocal_only_audio/hxu3KXVy48w_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/hwxTEcHnC1o_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/hBj0-dIU8HI_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/HL9kaJZw8iw_vocal.wav'
[Errno 2] No such file or directory: './resources/vocal_only_audio/0m5fIHHfJTw_vocal.wav'


/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


## Extract VGGish features

In [10]:
import pandas as pd
import torch
import numpy as np
import soundfile as sf
import os

model = torch.hub.load('harritaylor/torchvggish', 'vggish')
model.eval()

df=pd.DataFrame(np.load('./resources/working_data/vocal_only_data.npy',allow_pickle = True),columns=['video_id','start_time','mid_ts','label','audio'])
df['vggish']=''
for index,row in df.iterrows():
    file_name = './resources/working_data/'+f'block_{index}.wav'
    sf.write(file_name, row['audio'], 44100)
    vgg = model.forward(file_name)
    df['vggish'][index] = vgg.cpu().detach().numpy()
    if os.path.exists(file_name):
        os.remove(file_name)
    else:
        print("The file does not exist")
    
    # if index == 1:
    #     break

out = df.to_numpy()
np.save('./resources/working_data/vocal_only_data_with_vggish.npy', out)

Using cache found in /Users/vedant/.cache/torch/hub/harritaylor_torchvggish_master


## Extract feature vector

In [12]:
def block_audio(x,blockSize,hopSize,fs):    
    # allocate memory    
    numBlocks = math.ceil(x.size / hopSize)    
    xb = np.zeros([numBlocks, blockSize])    
    # compute time stamps    
    t = (np.arange(0, numBlocks) * hopSize) / fs   
    x = np.concatenate((x, np.zeros(blockSize)),axis=0)    
    for n in range(0, numBlocks):        
        i_start = n * hopSize        
        i_stop = np.min([x.size - 1, i_start + blockSize - 1])        
        xb[n][np.arange(0,blockSize)] = x[np.arange(i_start, i_stop + 1)]    
    return (xb,t)

def agg_mfccs(x):
    mfccs = librosa.feature.mfcc(x,n_mfcc = 13)
    mean = [np.mean(feature) for feature in mfccs]
    std = [np.std(feature) for feature in mfccs]
    mfcc_delta = librosa.feature.delta(mfccs)
    delta_mean=[np.mean(feature) for feature in mfcc_delta]
    delta_std=[np.std(feature) for feature in mfcc_delta]
    return mean,std,delta_mean,delta_std

def extract_features(x,fs=44100):
    #MFCCs
    mfcc_mean,mfcc_std,delta_mfcc_mean,delta_mfcc_std = agg_mfccs(x)
    #ZCR
    zcr=librosa.feature.zero_crossing_rate(x)
    #Spectral Centroid
    centroid = librosa.feature.spectral_centroid(x,44100)
    #Spectral Contrast
    contrast = librosa.feature.spectral_contrast(x,44100)
    #Spectral Flatness
    flatness = librosa.feature.spectral_flatness(x)
    #Spectral Roll-off
    rolloff = librosa.feature.spectral_rolloff(x,44100)
    #RMS
    rms = librosa.feature.rms(x)
    
    return mfcc_mean,mfcc_std,delta_mfcc_mean,delta_mfcc_std,zcr,centroid,contrast,flatness,rolloff,rms

df=pd.DataFrame(np.load('./resources/working_data/vocal_only_data_with_vggish.npy',allow_pickle = True),columns=['video_id','start_time','mid_ts','label','audio','vggish'])

df['zcr'] = ''
df['average_zcr'] = ''
df['zcr_stddev'] = ''

#df['mfccs'] = ''
df['mfcc_mean'] = ''
df['mfcc_std'] = ''

df['delta_mfcc_mean'] = ''
df['delta_mfcc_std'] = ''

df['centroid'] = ''
df['centroid_mean']=''
df['centroid_std'] = ''

df['contrast'] = ''
df['contrast_mean']=''
df['contrast_std'] = ''

df['flatness'] = ''
df['flatness_mean']=''
df['flatness_std'] = ''

df['rolloff'] = ''
df['rolloff_mean']=''
df['rolloff_std'] = ''

df['rms'] = ''
df['rms_mean']=''
df['rms_std'] = ''

#Row wise (block wise) aggregation of features by mean and std dev
for i in range(len(df)):
    #audio = butter_bandpass_filter(df['audio'][i],200,4500,44100)
    #audio = df['audio'][i]
    mean, std, delta_mean, delta_std, zcr, centroid, contrast, flatness, rolloff,rms = extract_features(df['audio'][i],44100)
    # Calculate ZCR - mean and std
    df['zcr'][i] = zcr
    df['average_zcr'][i] = np.mean(zcr)
    df['zcr_stddev'][i] = np.std(zcr)
    # Extract 13 MFCCs - get mean and std deviation for each (26 features) + Delta MFCCs (26 features) = total 52 Features
    df['mfcc_mean'][i] = mean
    df['mfcc_std'][i] = std
    df['delta_mfcc_mean'][i] = delta_mean
    df['delta_mfcc_std'][i] = delta_std

    #Calculate Spectral Centroid Mean and Std
    df['centroid'][i] = centroid
    df['centroid_mean'][i]=np.mean(centroid)
    df['centroid_std'][i] = np.std(centroid)

    #Calculate Spectral Contrast - Mean and Std
    df['contrast'][i] = contrast
    df['contrast_mean'][i]=np.mean(contrast)
    df['contrast_std'][i] = np.std(contrast)

    #Calculate spectral flatness - mean and std
    df['flatness'][i] = flatness
    df['flatness_mean'][i] = np.mean(flatness)
    df['flatness_std'][i] = np.std(flatness)

    #Calculate spectral flatness - mean and std
    df['rolloff'][i] = rolloff
    df['rolloff_mean'][i] = np.mean(rolloff)
    df['rolloff_std'][i] = np.std(rolloff)

    #Calculate rms - mean and std
    df['rms'][i] = rms
    df['rms_mean'][i] = np.mean(rms)
    df['rms_std'][i] = np.std(rms)


df[['mfcc1_mean','mfcc2_mean','mfcc3_mean','mfcc4_mean','mfcc5_mean','mfcc6_mean','mfcc7_mean','mfcc8_mean','mfcc9_mean','mfcc10_mean','mfcc11_mean','mfcc12_mean','mfcc13_mean']]=pd.DataFrame(df.mfcc_mean.tolist(), index= df.index)
df[['mfcc1_std','mfcc2_std','mfcc3_std','mfcc4_std','mfcc5_std','mfcc6_std','mfcc7_std','mfcc8_std','mfcc9_std','mfcc10_std','mfcc11_std','mfcc12_std','mfcc13_std']]=pd.DataFrame(df.mfcc_std.tolist(), index= df.index)

df[['delta_mfcc1_mean','delta_mfcc2_mean','delta_mfcc3_mean','delta_mfcc4_mean','delta_mfcc5_mean','delta_mfcc6_mean','delta_mfcc7_mean','delta_mfcc8_mean','delta_mfcc9_mean','delta_mfcc10_mean','delta_mfcc11_mean','delta_mfcc12_mean','delta_mfcc13_mean']]=pd.DataFrame(df.delta_mfcc_mean.tolist(), index= df.index)
df[['delta_mfcc1_std','delta_mfcc2_std','delta_mfcc3_std','delta_mfcc4_std','delta_mfcc5_std','delta_mfcc6_std','delta_mfcc7_std','delta_mfcc8_std','delta_mfcc9_std','delta_mfcc10_std','delta_mfcc11_std','delta_mfcc12_std','delta_mfcc13_std']]=pd.DataFrame(df.delta_mfcc_std.tolist(), index= df.index)

selected_cols=['video_id', 'start_time', 'mid_ts', 'label', 'average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std','vggish']
np.save('./resources/working_data/vocal_only_features.npy', df[selected_cols].to_numpy())